In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install sdv --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.3/154.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 9.1 MB/s eta 0:00:00


In [ ]:
import sdv
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
print(sdv.version.public)

1.17.3


In [ ]:
import sdv
print(sdv.__version__)

1.17.3


In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import TVAESynthesizer
from sdv.single_table import CopulaGANSynthesizer

from sdv.metadata import SingleTableMetadata

In [ ]:
import pickle
# Load object from file
with open(gdrivePath +os.sep + "data" +os.sep+'X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)


In [ ]:
print(X_train.shape,y_train.shape, X_test.shape,y_test.shape)

(11337, 7514) (11337,) (2835, 7514) (2835,)


Metadata

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

In [ ]:
X_train_pd = pd.DataFrame(X_train)    # values
             #index=data[1:,0],    # 1st column as index
             #columns=data[0,1:])

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(X_train_pd)
#metadata.set_primary_key(column_name='gene')

In [ ]:
metadata.visualize()

In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)
display(dict(zip(unique_values, counts)),np.max(counts))

{'PSC': 42,
 'acinar': 1090,
 'activated_stellate': 227,
 'alpha': 3897,
 'beta': 2950,
 'delta': 759,
 'ductal': 1360,
 'endothelial': 231,
 'epsilon': 17,
 'gamma': 339,
 'macrophage': 44,
 'mast': 20,
 'mesenchymal': 64,
 'pp': 148,
 'quiescent_stellate': 138,
 'schwann': 11}

3897

In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)
display(dict(zip(unique_values, counts)))
#counts

{'PSC': 42,
 'acinar': 1090,
 'activated_stellate': 227,
 'alpha': 3897,
 'beta': 2950,
 'delta': 759,
 'ductal': 1360,
 'endothelial': 231,
 'epsilon': 17,
 'gamma': 339,
 'macrophage': 44,
 'mast': 20,
 'mesenchymal': 64,
 'pp': 148,
 'quiescent_stellate': 138,
 'schwann': 11}

CTGAN

In [ ]:
def generate_ctgan(metadata,num_synthetic_samples, X_min):
    #start_time = time.strftime("%H:%M:%S", time.localtime())
    ctgan = CTGANSynthesizer(metadata,epochs=50,verbose=False)
    ctgan.fit(X_min)
    ctgan_synthetic = ctgan.sample(num_synthetic_samples)
    #ctgan_synthetic_df.to_csv(out_path2)
    #end_time = time.strftime("%H:%M:%S", time.localtime())
    #print('Total processing time for ctgan_synthesizer is : {:.5f}'.format(end_time - start_time) + " minutes")
    return ctgan_synthetic

In [ ]:
# Get unique values and their counts
unique_values, counts = np.unique(y_train, return_counts=True)
classlabel_counts = dict(zip(unique_values, counts))
max_count = 759 #np.max(counts)

i=1
for label, count in classlabel_counts.items():
    #print(x.shape, y.shape)
    print("label, count, max_count",label,count,max_count)
    X_minority = X_train_pd[y_train == label]
    if count < max_count:
        #print("\n")
        #print(f"Value {label} appears {count} times.")
        num_synthetic_samples = max_count - count
        synthetic_samples = generate_ctgan(metadata, num_synthetic_samples, X_minority)

        X_minority = np.array(X_minority)
        synthetic_samples = np.array(synthetic_samples)
        if i==1:
          X_train_gen = X_minority
          X_train_gen = np.vstack([X_train_gen, synthetic_samples])

          y_train_gen = np.full(759, label)

          y_train_indexes = np.full(count, 1)
          y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
        else:
          tmp = np.vstack([X_minority, synthetic_samples])
          X_train_gen = np.vstack([X_train_gen, tmp])
          y_train_gen = np.concatenate([y_train_gen, np.full(759, label)])

          y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
          y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
    else:
        X_train_gen = np.vstack([X_train_gen, X_minority])
        y_train_gen = np.concatenate([y_train_gen, np.full(count, label)])
        y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
    i=i+1


label, count, max_count PSC 42 759


/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:120: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Görüntülenen çıkış son 5000 satıra kısaltıldı.
EXOC3                  11
MYO1E                  11
CDH3                   11
CAMK1                  11
NYAP1                  11
ANAPC2                 11
MPL                    11
NLE1                   11
PCOLCE                 11
HELZ2                  11
C1orf198               11
SPSB3                  11
AVL9                   11
GATA4                  11
TTC39B                 11
AMY2B                  11
ZNF358                 11
F12                    11
ATP6V0C                11
TTC5                   11
NT5DC3                 11
ST3GAL1                11
MYCBP                  11
TPSAB1                 11
PPIF                   11
CNP                    11
ACTA1                  11
UNC5CL                 11
RBM38                  11
CD274                  11
TERF2IP                11
MAP3K5                 11
NPW                    11
CPNE2                  11
TTC39C                 11
ZNF607                 11
RAP2C            

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:120: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
CAMK1                  11
NYAP1                  11
ANAPC2                 11
MPL                    11
NLE1                   11
PCOLCE                 11
HELZ2                  11
C1orf198               11
SPSB3                  11
AVL9                   11
GATA4                  11
TTC39B                 11
AMY2B                  11
ZNF358                 11
F12                    11
ATP6V0C                11
TTC5                   11
NT5DC3                 11
ST3GAL1                11
MYCBP                  11
TPSAB1                 11
PPIF                   11
CNP                    11
ACTA1                  11
UNC5CL                 11
RBM38                  11
CD274                  11
TERF2IP                11
MAP3K5                 11
NPW                    11
CPNE2                  11
TTC39C                 11
ZNF607                 11
RAP2C                  11
FZR1                   11
TRAPPC4                11
ING3             

...

Görüntülenen çıkış son 5000 satıra kısaltıldı.
STK11                  11
EXOC3                  11
MYO1E                  11
CDH3                   11
CAMK1                  11
NYAP1                  11
ANAPC2                 11
MPL                    11
NLE1                   11
PCOLCE                 11
HELZ2                  11
C1orf198               11
SPSB3                  11
AVL9                   11
GATA4                  11
TTC39B                 11
AMY2B                  11
ZNF358                 11
F12                    11
ATP6V0C                11
TTC5                   11
NT5DC3                 11
ST3GAL1                11
MYCBP                  11
TPSAB1                 11
PPIF                   11
CNP                    11
ACTA1                  11
UNC5CL                 11
RBM38                  11
CD274                  11
TERF2IP                11
MAP3K5                 11
NPW                    11
CPNE2                  11
TTC39C                 11
ZNF607           

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:120: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
STK11                  11
EXOC3                  11
MYO1E                  11
CDH3                   11
CAMK1                  11
NYAP1                  11
ANAPC2                 11
MPL                    11
NLE1                   11
PCOLCE                 11
HELZ2                  11
C1orf198               11
SPSB3                  11
AVL9                   11
GATA4                  11
TTC39B                 11
AMY2B                  11
ZNF358                 11
F12                    11
ATP6V0C                11
TTC5                   11
NT5DC3                 11
ST3GAL1                11
MYCBP                  11
TPSAB1                 11
PPIF                   11
CNP                    11
ACTA1                  11
UNC5CL                 11
RBM38                  11
CD274                  11
TERF2IP                11
MAP3K5                 11
NPW                    11
CPNE2                  11
TTC39C                 11
ZNF607           

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:120: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name   Est # of Columns (CTGAN)
GCG                    11
INS                    11
TTR                    11
IAPP                   11
SST                    11
REG1A                  11
CHGB                   11
CTRB2                  11
CTRB1                  11
CELA3A                 11
SCG5                   11
TM4SF4                 11
PRSS1                  11
SPINK1                 11
SERPINA1               11
SERPINA3               11
CPE                    11
PCSK1N                 11
CHGA                   11
REG1B                  11
MALAT1                 11
CPA1                   11
RPS17                  11
SCG2                   11
CLU                    11
CLPS                   11
CELA3B                 11
VGF                    11
PLA2G1B                11
GPX3                   11
RNASEK                 11


In [ ]:
X_train_gen.shape, y_train_gen.shape, X_test.shape, y_test.shape

((18405, 7514), (18405,), (2835, 7514), (2835,))

In [ ]:
file_name ='ctGAN'

In [ ]:
import pickle

# Save object to file
with open(gdrivePath +os.sep+"results"+os.sep+"ctGAN"+os.sep+file_name+'_X_train.pkl', 'wb') as f:
    pickle.dump(X_train_gen, f)

with open(gdrivePath +os.sep+"results"+os.sep+"ctGAN"+os.sep+file_name+'_X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

with open(gdrivePath +os.sep+"results"+os.sep+"ctGAN"+os.sep+file_name+'_y_train.pkl', 'wb') as f:
    pickle.dump(y_train_gen, f)

with open(gdrivePath +os.sep+"results"+os.sep+"ctGAN"+os.sep+file_name+'_y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

with open(gdrivePath +os.sep+"results"+os.sep+"ctGAN"+os.sep+file_name+'_y_train_idx.pkl', 'wb') as f:
    pickle.dump(y_train_indexes, f)